In [3]:
%pylab inline
num_samples = 44100
input1 = linspace(-1, 1, num_samples)
input2 = linspace(1, -1, num_samples)

Populating the interactive namespace from numpy and matplotlib


In [4]:
input1[0], input1[-1]

(-1.0, 1.0)

In [6]:
!cat 01_Passthru.stride

use DesktopAudio version 1.0

AudioIn[1] >> AudioOut[1];
AudioIn[2] >> AudioOut[2];


In [7]:
out_text = ''
for val1, val2 in zip(input1, input2):
    out_text += str(val1) + "\n" + str(val2) + "\n"

out_file = open("01_Passthru.expected", "w")
out_file.write(out_text[:-1])
pass

In [8]:
!cat 02_Passthru_copy.stride

use DesktopAudio version 1.0

AudioIn[1] >> AudioOut[1];
AudioIn[1] >> AudioOut[2];


In [9]:
out_text = ''
for val1, val2 in zip(input1, input2):
    out_text += str(val1) + "\n" + str(val1) + "\n"

out_file = open("02_Passthru_copy.expected", "w")
out_file.write(out_text[:-1])
pass

In [10]:
!cat 03_Passthru_swap.stride

use DesktopAudio version 1.0

AudioIn[1] >> AudioOut[2];
AudioIn[2] >> AudioOut[1];


In [11]:
out_text = ''
for val1, val2 in zip(input1, input2):
    out_text += str(val2) + "\n" + str(val1) + "\n"

out_file = open("03_Passthru_swap.expected", "w")
out_file.write(out_text[:-1])
pass

In [12]:
!cat 04_Expressions_in_stream.stride

use DesktopAudio version 1.0

(AudioIn[1] * 0.5) + 1.0 >> AudioOut[1];
(AudioIn[2]/2) - 1.0 >> AudioOut[2];


In [13]:
out_text = ''
for val1, val2 in zip(input1, input2):
    out_text += str((val1* 0.5) + 1 ) + "\n" + str((val2/2) -1) + "\n"

out_file = open("04_Expressions_in_stream.expected", "w")
out_file.write(out_text[:-1])
pass

05 and 06 are copies of 01
~~~
cp 01_Passthru.expected 05_Passthru_bundle.expected
cp 01_Passthru.expected 06_Passthru_bundle_list.expected
~~~


In [14]:
cat 07_downsample.stride

use DesktopAudio version 1.0

signal DownSampled {
	rate: AudioRate/2
	default: 0.0
}

DownSampled + 1 >> DownSampled;

DownSampled >> AudioOut[1];


In [15]:
out_text = ''
counter = 0.5
for val1, val2 in zip(input1, input2):
    counter += 0.5
    out_text += str(int(counter)) + "\n0.0\n"

out_file = open("07_downsample.expected", "w")
out_file.write(out_text[:-1])
pass

In [16]:
cat 08_upsample.stride

use DesktopAudio version 1.0

signal Upsampled {
	rate: AudioRate*3
	default: 0.0
}

Upsampled + 1 >> Upsampled;

Upsampled >> AudioOut[1];


In [17]:
out_text = ''
counter = 0
for val1, val2 in zip(input1, input2):
    counter += 3
    out_text += str(int(counter)) + "\n0.0\n"

out_file = open("08_upsample.expected", "w")
out_file.write(out_text[:-1])
pass

In [18]:
cat 09_reset.stride

cat: 09_reset.stride: No such file or directory


In [19]:
out_text = ''
value = 1.0
for val1, val2 in zip(input1, input2):
    out_text += str(value) + "\n0.0\n"
    value += 1
    if value == 10:
        value = 1

out_file = open("09_reset.expected", "w")
out_file.write(out_text[:-1])
pass

In [20]:
cat 10_bundle_defaults.stride

use DesktopAudio version 1.0

signal Bundle[2] { default: [0.5, -0.5] }

AudioIn[1] * Bundle[1] >> AudioOut[1];
AudioIn[2] * Bundle[2] >> AudioOut[2];

In [21]:
out_text = ''
value = 1.0
for val1, val2 in zip(input1, input2):
    out_text += str(val1 * 0.5) + "\n" + str(val2 * -0.5) + "\n"

out_file = open("10_bundle_defaults.expected", "w")
out_file.write(out_text[:-1])
pass

In [22]:
cat 11_list_into_bundle.stride

use DesktopAudio version 1.0

import Generators

constant Val1 {value: 0.1}
constant Val2 {value: 0.2}
constant Val3 {value: 0.4}

[Val1, Val2, Val3] >> Bundle;

Bundle[1] >> AudioOut[1];
Bundle[2] + Bundle[3] >> AudioOut[2];

In [23]:
out_text = ''
value = 1.0
for val1, val2 in zip(input1, input2):
    out_text += "0.1\n0.6\n"

out_file = open("11_list_into_bundle.expected", "w")
out_file.write(out_text[:-1])
pass

In [24]:
cat 12_signal_list_into_bundle.stride

use DesktopAudio version 1.0

import Generators

signal Val1 {default: 0.1}
signal Val2 {default: 0.2}
signal Val3 {default: 0.4}

[Val1, Val2, Val3] >> Bundle;

Bundle[1] >> AudioOut[1];
Bundle[2] + Bundle[3] >> AudioOut[2];

In [26]:
out_text = ''
value = 1.0
for val1, val2 in zip(input1, input2):
    out_text += "0.1\n0.6\n"

out_file = open("12_signal_list_into_bundle.expected", "w")
out_file.write(out_text[:-1])
pass

In [27]:
cat 13_switch.stride

use DesktopAudio version 1.0

import Generators

switch Switch1 {}

AudioIn >> Greater() >> Switch1;
#Switch1 >> Select() >> AudioOut[1];

In [28]:
cat 14_trigger_reset.stride

use DesktopAudio version 1.0

trigger Reset { domain: AudioDomain }
	
signal Value {
	reset: Reset
	default: 1
    domain: AudioDomain
}

Value >> AudioOut[1];
Value + 1 >> Value;
[Value, 10] >> Equal() >> Reset;


In [29]:
out_text = ''
value = 1.0
for val1, val2 in zip(input1, input2):
    out_text += str(value) + "\n0.0\n"
    value += 1
    if value == 10:
        value = 1

out_file = open("14_trigger_reset.expected", "w")
out_file.write(out_text[:-1])
pass

In [1]:
cat 15_switch_reset.stride

use DesktopAudio version 1.0

switch Reset { domain: AudioDomain }
	
signal Value {
	reset: Reset
	default: 1
    domain: AudioDomain
}

Value >> AudioOut[1];
Value + 1 >> Value;
[Value, 10] >> Equal() >> Reset;


In [4]:
out_text = ''
value = 1.0
for val1, val2 in zip(input1, input2):
    out_text += str(value) + "\n0.0\n"
    value += 1
    if value == 10:
        value = 1

out_file = open("15_switch_reset.expected", "w")
out_file.write(out_text[:-1])
pass